## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [2]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [5]:
print(linkedin)

   
Contact
bertieliu52@gmail.com
www.linkedin.com/in/bhliu
(LinkedIn)
Top Skills
Site Reliability Engineering
Amazon Web Services (AWS)
iOS Development
Languages
English (Professional Working)
Finnish (Limited Working)
Chinese (Native or Bilingual)
Certifications
AWS Certified Solutions Architect –
Associate
PROFESSIONAL SCRUM MASTER
I
Magnolia Front-End Certificate Exam
Binghui Liu
Cloud, AI, Agile
Helsinki, Uusimaa, Finland
Experience
Accenture
4 years 7 months
Tech Architecture Associate Manager
June 2022 - Present (3 years 6 months)
Helsinki, Uusimaa, Finland
solution architect focused on cloud and AI, team leading and agile project
management
Application Specialist
May 2021 - June 2022 (1 year 2 months)
Helsinki, Southern Finland, Finland
I work in various projects both in private and public sectors. My responsibility
includes solution architecture, application development and project
management.
Qvik
3 years 7 months
Senior Software Engineer
June 2019 - May 2021 (2 years)
I have

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Binghui Liu"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Binghui Liu. You are answering questions on Binghui Liu's website, particularly questions related to Binghui Liu's career, background, skills and experience. Your responsibility is to represent Binghui Liu for interactions on the website as faithfully as possible. You are given a summary of Binghui Liu's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Binghui Liu. I'm a cloud architect, software engineer and agile master. I'm originally from Changchun, China, but I moved to Helsinki in 2014.\nI love to read books, both fictional and non-finctional. I also like to watch cute cat videos to kill time.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nbertieliu52@gmail.com\nwww.linkedin.com/in/bhliu\n(LinkedIn)\nTop Skills\nSite Reliability Engineering\nAmazon Web Services (AWS

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
import os
evaluator_openai = OpenAI()

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    response = evaluator_openai.responses.parse(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": evaluator_system_prompt},
            {"role": "user", "content": evaluator_user_prompt(reply, message, history)},
        ],
        text_format=Evaluation,
    )
    return response.output[0].content[0].parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [18]:
reply

'As of now, I do not hold a patent. My experience is primarily in software engineering, system architecture, cloud solutions, and agile project management. If you have any other questions about my background or skills, feel free to ask!'

In [24]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is clear and directly addresses the user's question about holding a patent. The Agent maintains a professional tone and offers additional information about Binghui Liu's experience, inviting further questions, which is engaging. Overall, the response is appropriate for the context of representing Binghui Liu on the website.")

In my free time, I enjoy reading a variety of books, both fictional and non-fictional, as it allows me to explore different ideas and perspectives. I also like to unwind by watching cute cat videos; it adds a bit of joy and lightness to my day. These activities provide a nice balance to my professional life and help me recharge!
Passed evaluation - returning reply


In [20]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content
    print(reply)

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In my free time, I enjoy reading both fictional and non-fictional books, which allows me to explore new ideas and broaden my knowledge. I also have a soft spot for watching cute cat videos—it's a fun and light-hearted way to unwind and relax. How about you? What do you enjoy doing in your free time?
Passed evaluation - returning reply
